In [5]:
import psycopg2
import pandas as pd

In [26]:
db_connection = psycopg2.connect(host="localhost",dbname="airflow",port=5532,user="postgres",password="postgres")

In [12]:
# df = pd.read_sql_query("SELECT from coingecko_historic WHERE timestamp >= 1678078800 AND timestamp < 1683345600 ", db_connection)

/var/folders/j2/8gptfrvd0_dfl_yxfcck_hh40000gn/T/ipykernel_42357/3463420340.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT from coingecko_historic WHERE timestamp >= 1678078800 AND timestamp < 1683345600 ", db_connection)


In [20]:
df = pd.read_sql_query("SELECT from coingecko_historic", db_connection)

/var/folders/j2/8gptfrvd0_dfl_yxfcck_hh40000gn/T/ipykernel_42357/3083386086.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT from coingecko_historic", db_connection)


In [21]:
df

""


In [18]:
params = {
    "host": "localhost",
    "dbname": "airflow",
    "port": 5532,
    "user": "postgres",
    "password": "postgres"
}
connection = None

try:
    connection = psycopg2.connect(**params)
    print("Connected to the database successfully.")
except Exception as e:
    print(f"An error occurred while connecting to the database: {e}")

# if connection:

#     df = pd.read_sql_query("SELECT * coingecko_historic", connection)

#     print(df)

#     connection.close()
# else:
#     print("Unable to create a DataFrame as the connection was not established.")


Connected to the database successfully.


In [22]:
import psycopg2
import pandas as pd

# Create a function that takes string input of a SQL query and output a dataframe contain the desired data.
def query_from_db(query):
    # Connection to the posrgreSQL in the airflow container.
    connection = psycopg2.connect(
            host="localhost",
            dbname="airflow",
            port=5532,
            user="postgres",
            password="postgres",
        )
    
    cursor = connection.cursor()
    cursor.execute(query)

    # Get the data from the cursor and make a dataframe for it.
    history_rows = cursor.fetchall()
    df = pd.DataFrame(history_rows)

    # Close the connection,
    cursor.close()
    connection.close()

    return df

In [23]:
query_from_db("SELECT from coingecko_historic WHERE timestamp >= 1678078800 AND timestamp < 1683345600 ")

""


In [24]:
def _create_insert_query(df: pd.DataFrame, db_table: str) -> str:
    columns = ",".join(list(df.columns))
    query = f"INSERT INTO {db_table}({columns}) VALUES %s"
    return query

In [25]:
def get_coingecko_historic_table_query() -> str:
    """
    Get SQL query to create coingecko table.
    """

    query = """
                CREATE TABLE IF NOT EXISTS coingecko_historic
                (
                    timestamp BIGINT,
                    price NUMERIC,
                    market_cap NUMERIC,
                    total_volume NUMERIC
                );
            """
    return query

In [27]:
cursor = db_connection.cursor()
        #
query = get_coingecko_historic_table_query()
cursor.execute(query)

In [30]:
from pycoingecko import CoinGeckoAPI

cg = CoinGeckoAPI()
data = cg.get_coin_market_chart_range_by_id(
id= 'bitcoin',
vs_currency= 'usd',
from_timestamp= 1678078800,
to_timestamp= 1683345600)

price = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
mc = pd.DataFrame(data['market_caps'], columns=['timestamp', 'market_cap'])
vol = pd.DataFrame(data['total_volumes'], columns=['timestamp', 'total_volume'])
price.set_index('timestamp', inplace=True)
mc.set_index('timestamp', inplace=True)
vol.set_index('timestamp', inplace=True)
df = pd.concat([price, mc, vol], axis=1)
df.reset_index(inplace=True)
print(df.columns)

Index(['timestamp', 'price', 'market_cap', 'total_volume'], dtype='object')


In [34]:
def _create_insert_query(df: pd.DataFrame, db_table: str) -> str:
        """
        Create an INSERT query to insert data into a DB.

        :param df: data to insert into DB
        :param table_name: name of the table for insertion
        :return: SQL query, e.g.,
            ```
            INSERT INTO coingecko_data (timestamp, price, market_cap, total_volume) VALUES %s
            ```
        """
        columns = ",".join(list(df.columns))
        query = f"INSERT INTO {db_table}({columns}) VALUES %s"
        return query

In [35]:
values = [tuple(v) for v in df.to_numpy()]

In [38]:
query = _create_insert_query(df, 'coingecko_historic')

In [40]:
import psycopg2.extras as extras
cursor = db_connection.cursor()
extras.execute_values(cursor, query, values)
db_connection.commit()

In [41]:
df

,timestamp,price,market_cap,total_volume
0,1678078892240,22362.008240,4.318985e+11,1.549735e+10
1,1678082525312,22375.039455,4.320736e+11,1.596562e+10
2,1678086057189,22422.125635,4.330265e+11,1.633610e+10
3,1678089776175,22413.812769,4.329764e+11,1.654128e+10
4,1678093343893,22431.198533,4.331373e+11,1.679919e+10
...,...,...,...,...
1459,1683327646632,29642.231839,5.736951e+11,7.941833e+09
1460,1683331228148,29520.322269,5.715202e+11,4.971524e+09
1461,1683334801988,29724.410733,5.759484e+11,1.262077e+10
1462,1683338435371,29583.277182,5.733244e+11,1.899716e+10


In [51]:
query_from_db("SELECT timestamp, price, market_cap, total_volume from coingecko_historic WHERE timestamp >= 1678078892240 AND timestamp < 1683342049799")

,0,1,2,3
0,1678078892240,22362.00823955405,431898530552.6892,15497348287.882515
1,1678082525312,22375.039455087837,432073636503.6055,15965619394.74735
2,1678086057189,22422.125634818287,433026518971.614,16336096947.327412
3,1678089776175,22413.812768997555,432976409654.98444,16541275982.752655
4,1678093343893,22431.198532520255,433137313807.4554,16799194209.257723
...,...,...,...,...
7310,1683324020324,29530.151331551937,571731122345.3936,18038758113.486103
7311,1683327646632,29642.231838723994,573695108243.3298,7941832613.386951
7312,1683331228148,29520.322268970496,571520224182.8824,4971523630.54862
7313,1683334801988,29724.410733072382,575948431256.4188,12620769574.944633
